<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#housekeeping" data-toc-modified-id="housekeeping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>housekeeping</a></span><ul class="toc-item"><li><span><a href="#languages-&amp;-packages" data-toc-modified-id="languages-&amp;-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>languages &amp; packages</a></span></li><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>env set-up</a></span></li></ul></li></ul></div>

# housekeeping
This notebook is for analytically solving the set of ODEs for eco-bioelectric cell (EBC).

## languages & packages
Julia-lang 1.3.1: `PyCall`  
python3 3.7.3: `SymPy`

## env set-up

In [1]:
using RCall, PyCall, DataFrames
mt = pyimport("math")
sc = pyimport("scipy")
itg = pyimport("scipy.integrate")
println()